In [1]:
import webbrowser
from aocd.models import Puzzle
puzzle = Puzzle(year=2019, day=5)
webbrowser.open(puzzle.url);

In [2]:
from dataclasses import dataclass
from typing import Callable
import numpy as np
from functools import partial

In [3]:
class ProgramStop(Exception):
    def __init__(self, final_memory):
        super().__init__()
        self.state = final_memory

@dataclass
class OpCode:
    name: str
    code: int
    n_args: int
    func: Callable

def exit(computer):
    raise ProgramStop(computer._memory)
    
def arithmetic(computer, op):
    computer.c = op(computer.a, computer.b)

def input_(computer):
    computer.a = computer.inputs.pop()

def output(computer):
    computer.outputs.append(computer.a)

def jump_if(computer, condition):
    if condition(computer.a):
        return computer.b

def arithmethic_comparison(computer, condition):
    computer.c = int(condition(computer.a, computer.b))

code_map = {
    op.code: op for op in [
        OpCode(name="add", code=1, n_args=3, func=partial(arithmetic, op=lambda a, b: a + b)),
        OpCode(name="multiply", code=2, n_args=3, func=partial(arithmetic, op=lambda a, b: a * b)),
        OpCode(name="input", code=3, n_args=1, func=input_),
        OpCode(name="output", code=4, n_args=1, func=output),
        OpCode(name="jump-if-true", code=5, n_args=2, func=partial(jump_if, condition=lambda a: a != 0)),
        OpCode(name="jump-if-false",code=6, n_args=2, func=partial(jump_if, condition=lambda a: a == 0)),
        OpCode(name="less than", code=7, n_args=3, func=partial(arithmethic_comparison, condition=lambda a, b: a < b)),
        OpCode(name="equals", code=8, n_args=3, func=partial(arithmethic_comparison, condition=lambda a, b: a == b)),
        OpCode(name="exit", code=99, n_args=0, func=exit)
    ]
}

def parse_opcode(code: int):
    """
    Parse the opcode and the parameter modes from the given opcode integer
    
    >>> parse_opcode(1002)
    OpCode('multiplay'), [0, 1, 0]
    
    >>> parse_opcode(1107)
    OpCode('less than'), [1, 1, 0]
    """
    instruction = code_map[code % 100]
    modes = [code // 10**(p+2) % 10 for p in range(instruction.n_args)]
    return instruction, modes

In [142]:
class Computer:
    def __init__(self, memory, inputs=None, debug=False):
        self.original_memory = memory
        self.original_inputs = [] if inputs is None else inputs
        self.debug = debug
    
    def simulate(self):
        self._memory = self.original_memory.copy()
        self.inputs = [a for a in self.original_inputs[::-1]]  # copy inputs
        self.outputs = []
        try:
            self._simulate()
            raise ValueError("Program did not terminate and instruction pointer ran out of bounds")
        except ProgramStop as stop:
            if self.debug:
                print("Program Terminated!")
            self._final_memory = stop.state
        return self.outputs
    
    def _simulate(self):
        self._ip = 0  # instruction pointer
        while self._ip < len(self._memory):
            self._instruction, self._param_modes = parse_opcode(self._memory[self._ip])
            self._params = self._memory[self._ip+1 : self._ip+1 + self._instruction.n_args]
            self._log_instruction_call_before()
            new_ip = self._instruction.func(self)
            self._log_instruction_call_after(new_ip)
            if new_ip is None:
                self._ip += 1 + self._instruction.n_args
            else:
                self._ip = new_ip
    
    # easy accessors for the parameters of the current instruction
    def _get_param(self, index):
        assert len(self._params) > index
        if self._param_modes[index] == 0:
            return self._memory[self._params[index]]
        return self._params[index]
    
    def _set_param(self, index, value):
        assert len(self._params) > index
        assert self._param_modes[index] == 0
        self._memory[self._params[index]] = value
    
    @property
    def a(self):
        return self._get_param(0)
    
    @a.setter
    def a(self, val):
        self._set_param(0, val)
    
    @property
    def b(self):
        return self._get_param(1)
    
    @b.setter
    def b(self, val):
        self._set_param(1, val)
    
    @property
    def c(self):
        return self._get_param(2)
    
    @c.setter
    def c(self, val):
        self._set_param(2, val)
    
    def _log_instruction_call_before(self):
        if not self.debug:
            return
        instr = ' '.join(map(str, self._memory[self._ip:self._ip+1 + self._instruction.n_args]))
        params = [f"[{par}]={self._memory[par]}" if m == 0 else str(par) for par, m in zip(self._params, self._param_modes)]
        print(f"{str(self._ip) + '  ':.<6} {instr+'  ':.<20}  {self._instruction.name+'  ':.<15}  {', '.join(params) + '  ':.<40}", end="")
    
    def _log_instruction_call_after(self, new_ip):
        if not self.debug:
            return
        writeable_params = set([par for par, m in zip(self._params, self._param_modes) if m == 0])
        values = [f"mem[{par}] = {self._memory[par]}" for par in writeable_params]
        if new_ip:
            print(f"✓ Instruction Pointer changed: New value {new_ip}")
        elif values:
            print(f"✓ Result: {', '.join(values)}")
        else:
            print("✓ No effect to memory or execution")

## Example

In [143]:
program = np.array([1002,4,3,4,33])
c = Computer(program, debug=True)
c.simulate()

0  ... 1002 4 3 4  ........  multiply  .....  [4]=33, 3, [4]=33  .....................✓ Result: mem[4] = 99
4  ... 99  ................  exit  .........    ......................................Program Terminated!


[]

## Part 1

In [144]:
program = np.array(list(map(int, puzzle.input_data.split(","))))
c = Computer(program, inputs=[1], debug=True)
outputs = c.simulate()
diagnostic_code = outputs[-1]
print(diagnostic_code)

0  ... 3 225  .............  input  ........  [225]=0  ...............................✓ Result: mem[225] = 1
2  ... 1 225 6 6  .........  add  ..........  [225]=1, [6]=1100, [6]=1100  ...........✓ Result: mem[225] = 1, mem[6] = 1101
6  ... 1101 1 238 225  ....  add  ..........  1, 238, [225]=1  .......................✓ Result: mem[225] = 239
10  .. 104 0  .............  output  .......  0  .....................................✓ No effect to memory or execution
12  .. 1101 32 43 225  ....  add  ..........  32, 43, [225]=239  .....................✓ Result: mem[225] = 75
16  .. 101 68 192 224  ....  add  ..........  68, [192]=92, [224]=0  .................✓ Result: mem[192] = 92, mem[224] = 160
20  .. 1001 224 -160 224  .  add  ..........  [224]=160, -160, [224]=160  ............✓ Result: mem[224] = 0
24  .. 4 224  .............  output  .......  [224]=0  ...............................✓ Result: mem[224] = 0
26  .. 102 8 223 223  .....  multiply  .....  8, [223]=0, [223]=0  ..............

In [146]:
assert diagnostic_code == 5821753

In [125]:
puzzle.answer_a = diagnostic_code

You don't seem to be solving the right level.  Did you already complete it? [Return to Day 5]


## Part 2

#### Tests

In [147]:
program = np.array([3,9,8,9,10,9,4,9,99,-1,8])
c = Computer(program, inputs=[8], debug=True)
outputs = c.simulate()
print(outputs[-1])

0  ... 3 9  ...............  input  ........  [9]=-1  ................................✓ Result: mem[9] = 8
2  ... 8 9 10 9  ..........  equals  .......  [9]=8, [10]=8, [9]=8  ..................✓ Result: mem[9] = 1, mem[10] = 8
6  ... 4 9  ...............  output  .......  [9]=1  .................................✓ Result: mem[9] = 1
8  ... 99  ................  exit  .........    ......................................Program Terminated!
1


In [148]:
program = np.array(list(map(int, puzzle.input_data.split(","))))
c = Computer(program, inputs=[5], debug=True)
outputs = c.simulate()
print(outputs[-1])

0  ... 3 225  .............  input  ........  [225]=0  ...............................✓ Result: mem[225] = 5
2  ... 1 225 6 6  .........  add  ..........  [225]=5, [6]=1100, [6]=1100  ...........✓ Result: mem[225] = 5, mem[6] = 1105
6  ... 1105 1 238  ........  jump-if-true  .  1, 238  ................................✓ Instruction Pointer changed: New value 238
238  . 1105 0 99999  ......  jump-if-true  .  0, 99999  ..............................✓ No effect to memory or execution
241  . 1105 227 247  ......  jump-if-true  .  227, 247  ..............................✓ Instruction Pointer changed: New value 247
247  . 1005 227 99999  ....  jump-if-true  .  [227]=0, 99999  ........................✓ Result: mem[227] = 0
250  . 1005 0 256  ........  jump-if-true  .  [0]=3, 256  ............................✓ Instruction Pointer changed: New value 256
256  . 1106 227 99999  ....  jump-if-false    227, 99999  ............................✓ No effect to memory or execution
259  . 1106 0 265  ....

In [149]:
assert outputs[-1] == 11956381

In [182]:
puzzle.answer_b = outputs[-1]

That's the right answer!  You are one gold star closer to rescuing Santa.You have completed Day 5! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
